In [1]:
import sys
import random
import binascii

In [2]:
class PrimeGenerator(object):

    def __init__(self, **kwargs):
        bits = debug = None
        if 'bits' in kwargs:
            bits = kwargs.pop('bits')
        if 'debug' in kwargs:
            debug = kwargs.pop('debug')

        self.bits = bits
        self.debug = debug
        self._largest = (1 << bits) -1

    def set_initial_candidate(self):
        candidate = random.getrandbits(self.bits)
        if candidate & 1 == 0:
            candidate += 1
        candidate |= (1 << self.bits-1)
        candidate |= (2 << self.bits-3)
        self.candidate = candidate

    def set_probes(self):
        self.probes = [2, 3, 5, 7, 11, 13, 17]

    def test_candidate_for_prime(self):
        p = self.candidate
        if p == 1:
            return 0
        if p in self.probes:
            self.probability_of_prime = 1
            return 1
        if any([p%a == 0 for a in self.probes]):
            return 0

        k, q = 0, self.candidate - 1
        while not q&1:
            q >>= 1
            k += 1
        if self.debug:
            print('q = ' + q + ' k = ' + k)

        for a in self.probes:
            a_raised_to_q = pow(a, q, p)
            if a_raised_to_q == 1 or a_raised_to_q == p-1:
                continue
            a_raised_to_jq = a_raised_to_q
            primeflag = 0
            for j in range(k-1):
                a_raised_to_jq = pow(a_raised_to_jq, 2, p)
                if a_raised_to_jq == p-1:
                    primeflag = 1
                    break
            if not primeflag:
                return 0
        self.probability_of_prime = 1 - 1.0/(4 ** len(self.probes))
        return self.probability_of_prime

    def findPrime(self):
        self.set_initial_candidate()
        if self.debug:
            print(" candidate is " + self.candidate)
        self.set_probes()
        if self.debug:
            print(" The probes are " + str(self.probes))

        max_reached = 0
        while 1:
            if self.test_candidate_for_prime():
                if self.debug:
                    print("Prime Number: %d with probability %f\n" %(self.candidate, self.probability_of_prime))
                break
            else:
                if max_reached:
                    self.candidate -= 2
                elif self.candidate >= self._largest - 2:
                    max_reached = 1
                    self.candidate -= 2
                else:
                    self.candidate += 2
                if self.debug:
                    print(" candidate is: %d" %self.candidate)
        return self.candidate

In [3]:
def MI(num, mod):
    x, x_old = 0, 1
    y, y_old = 1, 0
    store = mod

    while mod:
        q = num//mod
        num, mod = mod, num % mod
        x, x_old = x_old - q*x, x
        y, y_old = y_old - q*y, y

    if num != 1:
        print('No MI, gcd is ' + str(num))
    else:
        MI = x_old
        if MI < 0:
            MI = MI + store
        return MI

In [4]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('NO MI')
    else:
        return x % m

In [5]:
def genAll(e):
    gen = PrimeGenerator( bits = 128)
    p = 0
    q = 0
    d = 0
    while True:
        p = gen.findPrime()
        g, a, b = egcd(p-1,e)
        if(g == 1 and ((p & (3<<126)) == (3<<126))):
            break

    while True:
        q = gen.findPrime()
        g, a, b = egcd(q-1,e)
        if((g == 1) and p!=q and ((q & (3<<126)) == (3<<126))):
            break

    n = p*q
    phiOn = (p-1)*(q-1)
    d = MI(e, phiOn)
    return (p,q,n,phiOn,d)


In [53]:
def CRT(sum, e, n):
    res = 1
    c = 0
    b = bin(e)
    for i in range(0, len(b)):
        c = c*2
        res = (res*res) % n
        if(b[i] == '1'):
            c+=1
            res = (res*sum)%n
    return res

def decrypt(ci, p, q, n):
    vp = CRT(ci, d%(p-1), p)
    vq = CRT(ci, d%(q-1), q)
    xp = q * MI(q,p)
    xq = p * MI(p,q)
    M = (vp*xp + vq*xq)%n
    return M

In [57]:
file = open("plain.txt","r")
e = 65537
p, q, n, phiOn, d = genAll(e)
plaintext = file.read()
file.close()
cipher = ""
if((len(plaintext)%16) != 0):
    for i in range(16-(len(plaintext)%16)):
        plaintext += '\n'
for i in range(len(plaintext)//16):
    sum = 0
    for j in range(16):
        sum += ord(plaintext[(i*16)+j])
        if(j!=15):
            sum = sum<<8
    ci = CRT(sum,e,n)
    cipher += '{0:0256b}'.format(ci)
file1 = open("cipher.txt", "w")
file1.write(cipher)
file1.close()

In [58]:
file = open("cipher.txt","r")
cipher = file.read()
file.close()
plaintext = ""
for i in range(len(cipher)//256):
    c = cipher[i*256:(i+1)*256]
    dec = int(c,2)
    dec = decrypt(dec, p, q, n)
    plaintext += str(binascii.unhexlify('%x' %dec).decode('utf-8'))
print(plaintext)

With the explosive growth of internet, availability of cost-effective computing platforms and pervasive usages of resource-constrained devices, data nowadays are handled in many innovative ways. A little more.















